# Flask 설치 및 서버 실행하기

In [ ]:
from flask import Flask

# 초기화
app = Flask(__name__)

# 경로설정
@app.route("/")
def hello() :
    return "안녕안녕안녕"

# 메인 함수라면, py 파일을 단독으로 실행했다면
if __name__ == "__main__" :
    # 서버 시작
    app.run(host="192.168.0.98", port=9001)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://192.168.0.98:9001
Press CTRL+C to quit


# 웹 브라우저에 카메라 영상 출력하기

In [1]:
import cv2

cap = cv2.VideoCapture(0)

def get_frames() :
    while True :
        ret, frame = cap.read()
        
        frame = cv2.flip(frame, 1)
        
        cv2.waitKey(33)
        
        if not ret :
            continue
        # 이미지 프레임을 읽었다면 반환
        else :
            # 배열 jPG 변환해서 바이트단위로 전송
            ret, buff = cv2.imencode(".jpg", frame)
            frame = buff.tobytes()
            
            # 이미지를 반환
            yield (b'--frame\r\n' b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')
            
    cap.release()

- index.html를 작성하고 templates 폴더에 저장
  - video_feed : 실행할 함수명

- %%writefile : 아래에는 내용을 설정된 파일에 저장
    - 무조건 1줄에 써야된다.
- {{url_for('video_feed')}} : Jinja 템플릿
    - HTML 문서에 파이썬 코드를 작성하게 해주는 것

In [2]:
%%writefile ./templates/index.html
<html>
<body>
<h1>웹켐 테스트</h1>
<img src="{{url_for('video_feed')}}" width=50%>
</body>
</html>

Overwriting ./templates/index.html


In [ ]:
# render_template : Jinja 템플렛이 포함된 HTML을 전송
# Response : 클라이언트의 request에 대해서 응답을 해주는 기능
from flask import Flask, render_template, Response

app = Flask(__name__)

@app.route("/")
def index() :
    return render_template("index.html")

@app.route("/video_feed")
def video_feed() :
    return Response(get_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == "__main__" :
    app.run(host="192.168.0.98", port=9010 )

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://192.168.0.98:9010
Press CTRL+C to quit
192.168.0.98 - - [12/Oct/2023 17:22:24] "GET / HTTP/1.1" 200 -
192.168.0.98 - - [12/Oct/2023 17:22:25] "GET /video_feed HTTP/1.1" 200 -
